# Labo 7 : classification de textes (pour la désambiguïsation lexicale)
Nathan Gonzalez Montes & Guidoux Vincent

## Objectif et plan

L’objectif de ce laboratoire est d’utiliser des méthodes d’apprentissage supervisé pour classifier des occurrences du mot interest selon leur sens : c’est la même tâche, avec les mêmes données, que le labo 6. Pour classifier les occurrences, on considère leur contexte (mots voisins), et on applique l’approche bayésienne vue en cours : on entraîne un classifieur à six classes (les six sens de interest annotés de 1 à 6) sur une partie des données et on le teste sur la partie restante.

Dans ce laboratoire, on explore deux façons différentes de coder les traits (features) pour cette tâche. Dans les deux cas, on entraînera un NaiveBayesClassifier fourni par NLTK.1 Les deux façons sont :
1. Constituer un vocabulaire des mots qui apparaissent dans le voisinage de interest et définir ces mots comme traits. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme `{(‘rate’ : True), (‘in’ : False), … }` et on ajoute la classe (de 1 à 6).
2. Si word-1 est le mot précédant l’occurrence de interest, on définit comme traits word-n, …, word-2, word-1, word+1, word+2, …, word+n (une fenêtre de taille 2n autour de interest). Les valeurs possibles de ces traits sont cette fois-ci les mots observés, ou ‘NONE’ si la fenêtre dépasse les limites de la phrase. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme `{(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }` et on ajoute la classe (de 1 à 6).

Dans les deux cas, il faut diviser les 2368 occurrences de interest en un jeu d’entraînement et un jeu de test, en respectant la proportion initiale de chaque sens. Puis on entraîne un classifieur bayésien naïf en respectant le format de données indiqué par NLTK2, et on teste la performance du classifieur entraîné. L’objectif est de trouver les paramètres qui conduisent aux meilleurs scores de WSD.

## Importation

In [1]:
import nltk.tokenize

## Étapes proposées

### A. Traits lexicaux : présence ou absence de mots dans le voisinage de interest

#### 1. Le fichier de données se trouve à http://www.d.umn.edu/~tpederse/data.html – chercher « interest » vers la fin de la page, et prendre le fichier marqué comme « original format without POS tags » (le même qu’au labo 6). Lire le fichier et générer une liste de listes de mots (une liste par phrase) appelée `tokenized_sentences`.

In [2]:
filepath = 'data/interest-original.txt' 

try:  
    fp = open(filepath, 'r',encoding="utf-8")
    raw_sentences = fp.read().replace('=', '')
    raw_sentences = raw_sentences.split('\n$$\n')
finally:  
    fp.close()
    
tokenized_sentences = [nltk.word_tokenize(sent) for sent in raw_sentences]
    
tokenized_sentences = tokenized_sentences[:-1]

len(tokenized_sentences)

2368

In [3]:
tokenized_sentences

[['yields',
  'on',
  'money-market',
  'mutual',
  'funds',
  'continued',
  'to',
  'slide',
  ',',
  'amid',
  'signs',
  'that',
  'portfolio',
  'managers',
  'expect',
  'further',
  'declines',
  'in',
  'interest_6',
  'rates',
  '.'],
 ['longer',
  'maturities',
  'are',
  'thought',
  'to',
  'indicate',
  'declining',
  'interest_6',
  'rates',
  'because',
  'they',
  'permit',
  'portfolio',
  'managers',
  'to',
  'retain',
  'relatively',
  'higher',
  'rates',
  'for',
  'a',
  'longer',
  'period',
  '.'],
 ['nevertheless',
  ',',
  'said',
  'brenda',
  'malizia',
  'negus',
  ',',
  'editor',
  'of',
  'money',
  'fund',
  'report',
  ',',
  'yields',
  '``',
  'may',
  'blip',
  'up',
  'again',
  'before',
  'they',
  'blip',
  'down',
  '``',
  'because',
  'of',
  'recent',
  'rises',
  'in',
  'short-term',
  'interest_6',
  'rates',
  '.'],
 ['j.p.',
  'bolduc',
  ',',
  'vice',
  'chairman',
  'of',
  'w.r.',
  'grace',
  '&',
  'co.',
  ',',
  'which',
  'hol

#### 2. Définir une variable `window_size`, par exemple égale à 3 (on la fera varier plus tard), et une liste vide de mots `word_list`. Parcourir les `tokenized_sentences` et pour chaque phrase ajouter les mots voisins de interest (i.e. situés à une distance inférieure ou égale à `window_size`) dans la liste de mots `word_list`. Combien de mots contient celle-ci à la fin ? Tokens ou types ?

In [5]:
windows_size = 3
word_list = []

for sentence in (tokenized_sentences):
    for i, word in enumerate(sentence):
        print(i, word)

0 yields
1 on
2 money-market
3 mutual
4 funds
5 continued
6 to
7 slide
8 ,
9 amid
10 signs
11 that
12 portfolio
13 managers
14 expect
15 further
16 declines
17 in
18 interest_6
19 rates
20 .
0 longer
1 maturities
2 are
3 thought
4 to
5 indicate
6 declining
7 interest_6
8 rates
9 because
10 they
11 permit
12 portfolio
13 managers
14 to
15 retain
16 relatively
17 higher
18 rates
19 for
20 a
21 longer
22 period
23 .
0 nevertheless
1 ,
2 said
3 brenda
4 malizia
5 negus
6 ,
7 editor
8 of
9 money
10 fund
11 report
12 ,
13 yields
14 ``
15 may
16 blip
17 up
18 again
19 before
20 they
21 blip
22 down
23 ``
24 because
25 of
26 recent
27 rises
28 in
29 short-term
30 interest_6
31 rates
32 .
0 j.p.
1 bolduc
2 ,
3 vice
4 chairman
5 of
6 w.r.
7 grace
8 &
9 co.
10 ,
11 which
12 holds
13 a
14 83.4
15 %
16 interest_5
17 in
18 this
19 energy-services
20 company
21 ,
22 was
23 elected
24 a
25 director
26 .
0 finmeccanica
1 is
2 an
3 italian
4 state-owned
5 holding
6 company
7 with
8 interests_5
9 in
10 t

16 by
17 a
18 wide
19 margin
20 ``
21 and
22 it
23 noted
24 united
25 's
26 estimated
27 after-tax
28 loss
29 of
30 $
31 24
32 million
33 for
34 the
35 year
36 ended
37 june
38 30
39 .
0 although
1 takeover
2 experts
3 said
4 they
5 doubted
6 mr.
7 steinberg
8 will
9 make
10 a
11 bid
12 by
13 himself
14 ,
15 the
16 application
17 by
18 his
19 reliance
20 group
21 holdings
22 inc.
23 could
24 signal
25 his
26 interest_1
27 in
28 helping
29 revive
30 a
31 failed
32 labor-management
33 bid
34 .
0 younkers
1 management
2 is
3 likely
4 to
5 buy
6 a
7 10
8 %
9 to
10 20
11 %
12 interest_5
13 in
14 the
15 chain
16 in
17 january
18 ,
19 said
20 fred
21 s.
22 hubbell
23 ,
24 equitable
25 's
26 president
27 and
28 chief
29 executive
30 officer
31 .
0 reuters
1 holdings
2 plc
3 said
4 michael
5 reupke
6 resigned
7 as
8 general
9 manager
10 to
11 pursue
12 unspecified
13 interests_3
14 ,
15 a
16 move
17 the
18 news
19 organization
20 termed
21 an
22 ``
23 amicable
24 separation
25 .
0 the
1 company

5 wrote
6 to
7 complain
8 that
9 the
10 interest_6
11 he
12 was
13 paying
14 on
15 an
16 unsecured
17 $
18 10,000
19 loan
20 was
21 ``
22 absolutely
23 onerous
24 .
0 the
1 group
2 's
3 bid
4 for
5 $
6 40.9
7 million
8 of
9 wastewater
10 treatment
11 insured
12 bonds
13 ,
14 series
15 1989
16 a
17 ,
18 produced
19 a
20 7.0826
21 %
22 true
23 interest_6
24 cost
25 .
0 the
1 group
2 's
3 bid
4 for
5 $
6 34.2
7 million
8 of
9 wastewater
10 treatment
11 bonds
12 ,
13 series
14 1989
15 b
16 ,
17 produced
18 a
19 7.0808
20 %
21 true
22 interest_6
23 cost
24 .
0 interest_6
1 on
2 the
3 bonds
4 will
5 be
6 treated
7 as
8 a
9 preference
10 item
11 in
12 calculating
13 the
14 federal
15 alternative
16 minimum
17 tax
18 that
19 may
20 be
21 imposed
22 on
23 certain
24 investors
25 .
0 but
1 it
2 marks
3 a
4 big
5 change
6 for
7 the
8 japanese
9 ,
10 who
11 shunned
12 mortgage
13 securities
14 after
15 getting
16 burned
17 by
18 a
19 big
20 downturn
21 in
22 interest_6
23 rates
24 a
25 few
26 year

41 francs
42 a
43 year
44 earlier
45 .
0 for
1 all
2 of
3 1988
4 ,
5 bouygues
6 had
7 consolidated
8 profit
9 of
10 519
11 million
12 francs
13 ,
14 after
15 payments
16 to
17 minority
18 interests_5
19 ,
20 on
21 revenue
22 of
23 50
24 billion
25 francs
26 .
0 ``
1 the
2 interest_6
3 rate
4 sensitives
5 are
6 n't
7 rallying
8 with
9 the
10 rest
11 of
12 the
13 market
14 because
15 of
16 fears
17 about
18 what
19 the
20 (
21 federal
22 reserve
23 )
24 will
25 do
26 ,
27 ``
28 mr.
29 jennison
30 said
31 .
0 he
1 said
2 that
3 investors
4 will
5 scour
6 the
7 october
8 employment
9 report
10 ,
11 due
12 out
13 friday
14 ,
15 for
16 clues
17 about
18 the
19 direction
20 of
21 the
22 economy
23 and
24 the
25 immediate
26 outlook
27 for
28 interest_6
29 rates
30 .
0 the
1 separate
2 $
3 67
4 billion
5 bill
6 sent
7 to
8 the
9 white
10 house
11 had
12 budget
13 difficulties
14 ,
15 too
16 ,
17 but
18 was
19 saved
20 ultimately
21 by
22 its
23 importance
24 to
25 a
26 broad
27 spectrum
28 of


8 regular
9 weekly
10 treasury-bill
11 auction
12 .
0 here
1 are
2 details
3 of
4 yesterday
5 's
6 51-day
7 cash
8 management
9 bill
10 auction
11 :
12 interest_6
13 rate
14 8.07
15 %
16 the
17 bills
18 are
19 dated
20 oct.
21 31
22 and
23 mature
24 dec.
25 21
26 ,
27 1989
28 .
0 in
1 june
2 ,
3 western
4 union
5 was
6 forced
7 to
8 reset
9 the
10 interest_6
11 rate
12 on
13 the
14 senior
15 secured
16 notes
17 due
18 in
19 1992
20 to
21 19
22 14
23 %
24 from
25 16
26 12
27 %
28 ,
29 a
30 move
31 which
32 increased
33 the
34 firm
35 's
36 annual
37 *interest
38 payments
39 by
40 $
41 13.8
42 million
43 .
0 in
1 june
2 ,
3 western
4 union
5 was
6 forced
7 to
8 reset
9 the
10 *interest
11 rate
12 on
13 the
14 senior
15 secured
16 notes
17 due
18 in
19 1992
20 to
21 19
22 14
23 %
24 from
25 16
26 12
27 %
28 ,
29 a
30 move
31 which
32 increased
33 the
34 firm
35 's
36 annual
37 interest_6
38 payments
39 by
40 $
41 13.8
42 million
43 .
0 ``
1 my
2 impression
3 is
4 that
5 there
6 is
7 n't
8

...

4 do
5 n't
6 know
7 of
8 any
9 way
10 to
11 protect
12 all
13 u.s.
14 interests_5
15 in
16 such
17 countries
18 as
19 bolivia
20 ,
21 peru
22 and
23 colombia
24 ,
25 where
26 the
27 cartels
28 '
29 power
30 is
31 immense
32 .
0 the
1 junk-bond
2 funds
3 continue
4 to
5 rake
6 in
7 enormous
8 interest_6
9 payments
10 .
0 investors
1 have
2 racked
3 up
4 a
5 measly
6 5.7
7 %
8 total
9 return
10 ,
11 or
12 price
13 changes
14 plus
15 interest_6
16 payments
17 ,
18 for
19 a
20 merrill
21 lynch
22 junk-bond
23 index
24 in
25 the
26 year
27 to
28 date
29 .
0 the
1 buy-back
2 ,
3 at
4 par
5 plus
6 accrued
7 interest_6
8 ,
9 would
10 occur
11 within
12 12
13 months
14 after
15 the
16 planned
17 restructuring
18 of
19 campeau
20 and
21 its
22 subsidiaries
23 is
24 completed
25 .
0 but
1 uncertainty
2 regarding
3 the
4 direction
5 of
6 currencies
7 and
8 domestic
9 interest_6
10 rates
11 discouraged
12 further
13 buying
14 *interest
15 ,
16 traders
17 said
18 .
0 but
1 uncertainty
2 regarding
3 

11 bell
12 resources
13 to
14 buy
15 back
16 about
17 a
18 $
19 600
20 million
21 of
22 bonds
23 it
24 has
25 issued
26 that
27 are
28 convertible
29 into
30 bell
31 resources
32 shares
33 ,
34 and
35 to
36 arrange
37 funds
38 to
39 repurchase
40 an
41 undisclosed
42 amount
43 of
44 subordinated
45 debentures
46 issued
47 by
48 bond
49 corp.
50 brewing
51 interests_5
52 .
0 the
1 merc
2 alleged
3 that
4 siegel
5 trading
6 ``
7 favored
8 its
9 own
10 rather
11 than
12 customers
13 '
14 best
15 interests_4
16 ``
17 by
18 making
19 trades
20 in
21 customers
22 '
23 accounts
24 without
25 their
26 approval
27 .
0 tuesday
1 ,
2 september
3 19
4 ,
5 1989
6 the
7 key
8 u.s.
9 and
10 foreign
11 annual
12 interest_6
13 rates
14 below
15 are
16 a
17 guide
18 to
19 general
20 levels
21 but
22 do
23 n't
24 always
25 represent
26 actual
27 transactions
28 .
0 relatively
1 small
2 groups
3 organized
4 around
5 narrowly
6 focused
7 interests_3
8 find
9 advantage
10 in
11 lobbying
12 government
13 for

13 billion
14 bond
15 issue
16 mr.
17 holmes
18 a
19 court
20 's
21 bell
22 resources
23 ltd.
24 would
25 have
26 used
27 to
28 reduce
29 its
30 interest_6
31 bill
32 .
0 ``
1 the
2 idea
3 was
4 he
5 might
6 make
7 it
8 ,
9 ``
10 one
11 banker
12 says
13 ,
14 but
15 ``
16 high
17 interest_6
18 rates
19 changed
20 a
21 lot
22 .
0 david
1 maxwell
2 ,
3 fannie
4 mae
5 's
6 chairman
7 ,
8 said
9 the
10 split
11 and
12 the
13 dividend
14 increase
15 ``
16 reflect
17 our
18 confidence
19 in
20 fannie
21 mae
22 's
23 continued
24 success
25 and
26 our
27 interest_1
28 in
29 broadening
30 the
31 distribution
32 of
33 our
34 stock
35 .
0 the
1 company
2 said
3 thomas
4 csathy
5 ,
6 its
7 president
8 and
9 chief
10 operating
11 officer
12 since
13 1986
14 ,
15 has
16 resigned
17 ``
18 to
19 pursue
20 other
21 interests_3
22 .
0 the
1 notion
2 stems
3 from
4 sen.
5 bentsen
6 's
7 interest_1
8 in
9 encouraging
10 long-range
11 thinking
12 by
13 american
14 business
15 *interests
16 and
17 in
18 di

17 to
18 make
19 an
20 interest_6
21 payment
22 last
23 week
24 prompted
25 much
26 of
27 the
28 decline
29 in
30 junk
31 bond
32 prices
33 .
0 mitsui
1 also
2 has
3 an
4 option
5 to
6 purchase
7 another
8 5
9 %
10 interest_5
11 later
12 .
0 the
1 utility
2 holding
3 company
4 said
5 the
6 redemption
7 is
8 expected
9 to
10 save
11 more
12 than
13 $
14 2
15 million
16 in
17 interest_6
18 costs
19 over
20 the
21 next
22 two
23 years
24 .
0 it
1 will
2 redeem
3 them
4 at
5 face
6 amount
7 plus
8 one
9 day
10 's
11 accrued
12 and
13 unpaid
14 interest_6
15 from
16 oct.
17 15
18 to
19 the
20 redemption
21 date
22 .
0 accrued
1 semiannual
2 interest_6
3 due
4 oct.
5 15
6 will
7 be
8 paid
9 to
10 holders
11 of
12 record
13 oct.
14 1
15 .
0 the
1 japanese
2 securities
3 house
4 already
5 has
6 a
7 25.1
8 %
9 interest_5
10 in
11 matuschka
12 's
13 mat
14 main
15 anlage
16 trust
17 g.m.b
18 .
0 the
1 new
2 hippocrates
3 will
4 have
5 editorial
6 matter
7 similar
8 to
9 in
10 health
11 ,
12 but


6 retail
7 sales
8 ,
9 while
10 manufacturing
11 inventories
12 are
13 growing
14 .
0 ``
1 if
2 these
3 problems
4 are
5 cropping
6 up
7 now
8 with
9 modest
10 inflation
11 ,
12 interest_6
13 rates
14 down
15 and
16 the
17 economy
18 expanding
19 ,
20 what
21 in
22 god
23 's
24 name
25 do
26 you
27 think
28 will
29 happen
30 with
31 extremely
32 tight
33 money
34 and
35 an
36 economic
37 downturn
38 .
0 the
1 company
2 said
3 others
4 besides
5 investor
6 arthur
7 goldberg
8 ,
9 who
10 is
11 bidding
12 $
13 32
14 a
15 share
16 ,
17 have
18 expressed
19 interest_1
20 in
21 buying
22 all
23 or
24 part
25 of
26 its
27 operations
28 .
0 analysts
1 say
2 the
3 weakness
4 in
5 the
6 junk-bond
7 market
8 is
9 reducing
10 interest_1
11 in
12 many
13 takeover
14 stocks
15 .
0 traders
1 say
2 a
3 favorable
4 article
5 on
6 the
7 company
8 's
9 orgotein
10 painkiller
11 in
12 the
13 current
14 issue
15 of
16 the
17 american
18 journal
19 of
20 medicine
21 has
22 spurred
23 interest_1
24 in
25 the

#### 3. À l’aide d’un objet NLTK de type FreqDist, sélectionner parmi les mots de word_list les N plus fréquents, dans une nouvelle liste appelée vocabulary (p.ex. N = 500, mais on le fera varier). Affichez les 50 mots les plus fréquents. Est-ce une bonne idée d’enlever les stopwords ?

#### 4. Parcourir à nouveau les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une liste appelée feature_sets.

##### 4.1 Prendre modèle sur https://www.nltk.org/book/ch06.html (début du 1.2)

##### 4.2 Pour le dictionnaire, il faut créer un trait pour chaque mot de vocabulary, et examiner si ce mot est présent dans une fenêtre de taille window_size autour de l’occurrence de interest : si oui, le trait est True, sinon il est False. Par exemple, on aboutit à : {'contains(the)': False, 'contains(,)': True, 'contains(rates)': True, …}.

##### 4.3 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

##### 4.3 Combien d’occurrences pour chaque sens de interest y a-t-il dans feature_sets ?

#### 5. Diviser les données de feature_sets en deux sous-ensembles : l’un comportant 80% des données est le train_set, et l’autre (20%) est le test_set. Attention, il faut respecter deux conditions :

##### 5.1 Chaque sens doit être présent dans les mêmes proportions dans train_set et dans test_set (donc il faut faire la division de manière séparée pour chaque sens).

##### 5.2 Mélanger avec shuffle() les occurrences avant de prendre les 80% premières pour le `train_set`et les 20% restantes dans le test_set.

#### 6. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set, puis le tester sur les données de test_set. Quelle est la précision (accuracy) atteinte ?

#### 7. Adapter le code précédent pour effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

#### 8. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. 1, 3, 5, 7, 11) et la taille du vocabulaire (50, 100, 200, 500, 1000 mots). Combien d’expériences faut-il exécuter ? Quelle est la meilleure combinaison fenêtre x vocabulaire et quel est le score moyen obtenu ?

### B. Traits lexicaux positionnels : valeurs des mots précédant/suivant interest
Pour cette deuxième partie, on réutilisera beaucoup d’éléments de la première. Seule la nature des
traits utilisés et leur extraction vont changer.

#### 1. Partir de la liste de listes de mots (une liste par phrase) précédente, appelée tokenized_sentences.

#### 2. Définir une variable window_size2, par exemple égale à 3 (on la fera varier plus tard).

#### 3. Parcourir les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une nouvelle liste appelée feature_sets2.

##### 3.1 Pour le dictionnaire de traits, il faut cette fois-ci créer un trait pour chaque position relative par rapport à interest, donc ‘word-1’, ‘word+1’, etc. (jusqu’à window_size2). La valeur du trait sera le mot trouvé à cette position, ou ‘NONE’ si on sort de la phrase. Par exemple {(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }.

##### 3.2 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

#### 4. Diviser les données de feature_sets2 en deux sous-ensembles (80%/20%) appelés train_set2 et test_set2 avec la même procédure qu’à la partie A.

#### 5. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set2, puis le tester sur les données de test_set2. Quelle est la précision (accuracy) atteinte ?

#### 6. Effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

#### 7. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. entre 1 et 15). Quelle est la meilleure valeur et quel est le score moyen obtenu ?

#### 8. Quelle est le meilleur score obtenu entre (A) et (B) ?

#### 9. Thème de réflexion facultatif : les différences des scores sont-elles statistiquement significatives ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le **lundi 27 mai à 23h59**.